# Spatial convergence test

In [ ]:
from pathlib import Path
import json
import numpy as np
from collections import defaultdict
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests

pd.options.display.max_columns = 5

In [ ]:
headers = {
    "Accept": "application/vnd.github+json",
    "X-GitHub-Api-Version": "2022-11-28",
}
print("Get existing data from gist")
response = requests.get(
    f"https://api.github.com/gists/73fa6531f28da2b3633a7ddaca38a7cd",
    headers=headers,
)
data = json.loads(response.json()["files"]["convergence_test.json"]["content"])

In [ ]:
runs = set()
all_results = defaultdict(list)
spatial_convergence = []
for git_hash, d in data.items():
    runs |= set(d.keys())
    if "dx0.1_dt0.05" not in d:
        # We have not spatial convergence data
        continue

    for key1, d1 in d.items():
        if "dt0.05" not in key1:
            continue  # Only select data with "dt0.05"
        for key, value in d1.items():
            all_results[key].append(value)

N = max(len(v) for v in all_results.values())
results = {}
for k, v in all_results.items():
    print(k, len(v))
    if len(v) == N:
        results[k] = v

df = pd.DataFrame(results)
df["timestamp"] = pd.to_datetime(df["timestamp"])
df

In [ ]:
df["timestamp"]
df.keys()
df["dx"].unique()

In [ ]:
def get_ylim(values):
    if np.isclose(values, 1).all():
        return (0.99, 1.01)
    y_max = np.max(values)
    y_min = np.min(values)
    d_max = y_max - 1
    d_min = 1 - y_min
    
    d = max(d_min, d_max) + 0.025
    return (1 - d, 1 + d)


In [ ]:
dxs = df["dx"].unique()
DX = np.median(dxs)   # Choose the dx in the middle as the one to compare with
dxs

In [ ]:
columns = [c for c in df.columns if c not in ["timestamp", "simcardems_version", "sha", "dx", "dt"]]
git_hash = df["sha"]
versions = df["simcardems_version"]
dxs = df["dx"].unique()
dates = [t.date() for t in df["timestamp"]]

df_DX = df[df["dx"] == DX]
df_DX.sort_values(by='timestamp')
    
fig = make_subplots(
    rows=len(columns), 
    cols=1, 
    subplot_titles=columns,  
    y_title="Percentage deviation from median case", 
    shared_xaxes=True
)

yranges = []
colors = ['red', 'green', 'blue']
for color, dx in zip(colors, sorted(dxs)):
    df_dx = df[df["dx"] == dx]
    df_dx.sort_values(by='timestamp')
    
    git_hash = df_dx["sha"]
    versions = df_dx["simcardems_version"]
    dates = [t.date() for t in df_dx["timestamp"]]
    
    text = []
    for h,v, t in zip(git_hash, versions, dates):
        text.append("\n".join([
            f"<br>Git Hash: {h}</br>",
            f"<br>Version: {v}</br>",
            f"<br>Timestamp {t}</br>",
        ]))
 
    for i, c in enumerate(columns):
        
        y = df_dx[c].to_numpy() / df_DX[c].to_numpy()
        
        row = i + 1
        col = 1
        showlegend=i==0
        fig.add_trace(
            go.Scatter(
                x=df_dx["timestamp"], 
                y=y, 
                text=text, 
                hovertemplate="%{text}",
                name=f"dx={dx}", 
                legendgroup=str(dx),
                showlegend=i==0,
                marker_color=color
            ),
            row=row, col=col
        )
        fig.update_yaxes(range=get_ylim(y), row=row, col=col)

fig.update_layout(
    height=5000,
    showlegend=True
)
fig.show()